# Check whether two fans have the same support

Preconditions: Both fans are pure and full-dimensional.

Note: This notebook is written to be understandable and work on small examples. It is by no means tuned for high performance, and it will most likely perform badly on big examples.

In [1]:
application "fan";

We will proceed by example. Let us first make three fans, two of which have the same support.

In [2]:
$f1 = new PolyhedralFan(INPUT_RAYS=>[[1,0],[-1,-1],[0,1]], INPUT_CONES=>[[0,1],[1,2]]);
$f2 = new PolyhedralFan(INPUT_RAYS=>[[1,0],[0,-1],[-1,0],[0,1]], INPUT_CONES=>[[0,1],[1,2],[2,3]]);
$f3 = normal_fan(cube(2));

The fans `$f1` and `f2` have the same support, which is the whole twodimensional space, without the positive orthant. In contrast, the fan `$f3` is complete.

## Comparing `$f1` and `$f2`

The first step is to compute the common refinement.

In [3]:
$f12 = common_refinement($f1, $f2);

Click here for additional output 
 
polymake: used package cdd
 cddlib
 Implementation of the double description method of Motzkin et al.
 Copyright by Komei Fukuda.
 http://www-oldurls.inf.ethz.ch/personal/fukudak/cdd_home/

A simple first check is to see whether the common refinement is pure and full-dimensional too:

In [4]:
print "Pure? ", $f12->PURE, "\n";
print "Full-dim? ", $f12->FULL_DIM, "\n";

Pure? true
Full-dim? true


Now the general procedure is easy to describe: We need to check that the common refinement covers all the cones of `$f1` and `$f2`. Since this is symmetric, we will only check for `$f1`. And we will only do so for the first maximal cone of `$f1`, before we assemble a general method.

In [5]:
$c1 = $f1->cone(0);

Now select those cones of `$f12` contained in `$c1`:

In [6]:
@cover = grep(included_polyhedra($f12->cone($_), $c1), 0..($f12->N_MAXIMAL_CONES-1));
print join(",", @cover), "\n";

0,1


Next we write a small helper method that constructs us the facets of a cone as cones:

In [7]:
sub facets_of_cone {
    my $c = shift;
    my @cfacets = @{$c->FACETS_THRU_RAYS};
    my $R = $c->RAYS;
    my $L = $c->LINEALITY_SPACE;
    return map(new Cone(INPUT_RAYS=>$R->minor($_, All), INPUT_LINEALITY=>$L), @cfacets);
}

The idea is now to collect all the facets of the covering cones, remove those that are contained in the facets of the original `$c1` and then checking that the remaining facets appear exactly twice. This is similar to the algorithm for checking completeness of a fan.

In [8]:
@covercones = map($f12->cone($_), @cover);
@coverfacets = map(facets_of_cone($_), @covercones);
@c1facets = facets_of_cone($c1);
print "Total number of facets: ", scalar(@coverfacets), "\n";

Total number of facets: 4


In [9]:
foreach my $facet (@c1facets){
    @coverfacets = grep(!equal_polyhedra($_, $facet), @coverfacets);
}
print "There are ", scalar(@coverfacets), " facets left\n";

There are 2 facets left


In [10]:
for(my $i=0; $i<scalar(@coverfacets)-1; $i++){
    my $equals = 0;
    my $facet1 = $coverfacets[$i];
    for(my $j=$i+1; $j<scalar(@coverfacets); $j++){
        my $facet2 = $coverfacets[$j];
        if(equal_polyhedra($facet1, $facet2)){
            $equals++;
        }
    }
    print "Facet $i has $equals copies\n";
}

Facet 0 has 1 copies


## General method

We assemble the previous methods to give a general function.

In [11]:
sub facets_of_cone {
    my $c = shift;
    my @cfacets = @{$c->FACETS_THRU_RAYS};
    my $R = $c->RAYS;
    my $L = $c->LINEALITY_SPACE;
    return map(new Cone(INPUT_RAYS=>$R->minor($_, All), INPUT_LINEALITY=>$L), @cfacets);
}

sub cone_is_covered {
    my($cone, $fan) = @_;
    my @cover = grep(included_polyhedra($fan->cone($_), $cone), 0..($fan->N_MAXIMAL_CONES-1));
    if(scalar(@cover) == 0){
        # If there are no cones for covering then we must return false
        return false;
    }
    my @covercones = map($fan->cone($_), @cover);
    my @coverfacets = map(facets_of_cone($_), @covercones);
    my @conefacets = facets_of_cone($cone);
    foreach my $facet (@conefacets){
        @coverfacets = grep(!equal_polyhedra($_, $facet), @coverfacets);
    }
    for(my $i=0; $i<scalar(@coverfacets)-1; $i++){
        my $equals = 0;
        my $facet1 = $coverfacets[$i];
        for(my $j=$i+1; $j<scalar(@coverfacets); $j++){
            my $facet2 = $coverfacets[$j];
            if(equal_polyhedra($facet1, $facet2)){
                $equals++;
            }
            if($equals > 1){
                return false;
            }
        }
        if($equals != 1){
            return false;
        }
    }
    return true;
}

sub fan_is_covered {
    my($fan, $cover) = @_;
    for(my $i=0; $i<$fan->N_MAXIMAL_CONES; $i++){
        my $cone = $fan->cone($i);
        if(!cone_is_covered($cone, $cover)){
            return false;
        }
    }
    return true;
}

sub equal_support{
    my($f1, $f2) = @_;
    # Not checking that $f1 and $f2 satisfy preconditions.
    my $f12 = common_refinement($f1, $f2);
    if(!fan_is_covered($f1, $f12)){
        return false;
    }
    return fan_is_covered($f2, $f12);
}

In [12]:
print "\$f1 and \$f2: ", equal_support($f1, $f2), "\n";
print "\$f1 and \$f3: ", equal_support($f1, $f3), "\n";
print "\$f2 and \$f3: ", equal_support($f2, $f3), "\n";

$f1 and $f2: true
$f1 and $f3: false
$f2 and $f3: false
